In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import wandb
epochs = 20

In [2]:
def set_seed(seed = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [3]:
wandb.init(project="CNN on MNIST pytorch", save_code=True,
           config={
               "batch_size": 128,
               "epochs": 20,
               "optimizer": "Adam",
               "loss function": "CrossEntropyLoss",
           }
           )

config = wandb.config

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\SMAYA\_netrc.
wandb: Currently logged in as: b25cs1065 (b25cs1065-indian-institute-of-technology-jodhpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using device {device}")

using device cuda:0


In [5]:
#Prepare the MNIST dataset in this cell
''
train_transform = transforms.Compose([transforms.ToTensor(),
                                transforms.RandomRotation(degrees=10),
                                transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.7, 1.3)),
                                transforms.Normalize((0.5,), (0.5,))])
''
test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=test_transform)
print(f"training data shape: {trainset.data.shape}")
print(f"training labels shape: {trainset.targets.shape}")
print(f"testing data shape: {testset.data.shape}")
print(f"testing labels shape: {testset.targets.shape}")
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle = True)

training data shape: torch.Size([60000, 28, 28])
training labels shape: torch.Size([60000])
testing data shape: torch.Size([10000, 28, 28])
testing labels shape: torch.Size([10000])


In [6]:
class Neural_network(nn.Module):
    def __init__(self):
        super(Neural_network, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)#output dim = 14x14x8
        self.conv2 = nn.Conv2d(16, 28, 3, padding =1)#output dim = 7x7x16
        #self.dropout = nn.Dropout(p=0.15)
        self.normalization1 = nn.BatchNorm2d(16)
        self.normalization2 = nn.BatchNorm2d(28)
        self.normalization3 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(28, 32, 3, padding =1)#output dim = 3x3x20
        #self.fc1 = nn.Linear(4*4*32, 16)
        self.fc2 = nn.Linear(32, 10)
        #self.fc3 = nn.Linear(16, 10)
        self.pool = nn.MaxPool2d(2, 2)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
    def forward(self,x):
        x= self.pool(F.leaky_relu(self.normalization1(self.conv1(x))))
        x= self.pool(F.leaky_relu(self.normalization2(self.conv2(x))))
        x= F.leaky_relu(self.normalization3(self.conv3(x)))
        #print(f"Shape after conv3 and pool: {x.shape}")
        x = self.gap(x)
        x= x.view(-1, 32)
        #x= F.leaky_relu(self.fc1(x))
        #x = self.dropout(x)
        x = self.fc2(x)
        #x = self.fc3(x)
        return x
    
net = Neural_network()
net.to(device)

Neural_network(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 28, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (normalization1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (normalization2): BatchNorm2d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (normalization3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(28, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc2): Linear(in_features=32, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (gap): AdaptiveAvgPool2d(output_size=(1, 1))
)

In [7]:
total_params = sum(p.numel() for p in net.parameters())
print(f"Total number of parameters in the model: {total_params}")

Total number of parameters in the model: 12798


In [8]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)#does net.parameters just make a vector of all the parameters in the model?

In [9]:
print(f"{trainloader}")

In [10]:
#have to decide how to train the model in this cell ie epochs, batch size, backpropagation etc.
'''
epochs = 20
for epoch in range(epochs):
    dynamic_loss = 0.0
    for i,data in enumerate(trainloader):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, F.one_hot(labels, num_classes=10).float())
        loss.backward()
        optimizer.step()
        dynamic_loss += loss
        if i%200 ==99:
            print(f"epoch {epoch+1}, batch {i+1}, loss: {dynamic_loss/400}")
            dynamic_loss = 0.0 
'''

'\nepochs = 20\nfor epoch in range(epochs):\n    dynamic_loss = 0.0\n    for i,data in enumerate(trainloader):\n        inputs, labels = data[0].to(device), data[1].to(device)\n        optimizer.zero_grad()\n        outputs = net(inputs)\n        loss = loss_function(outputs, F.one_hot(labels, num_classes=10).float())\n        loss.backward()\n        optimizer.step()\n        dynamic_loss += loss\n        if i%200 ==99:\n            print(f"epoch {epoch+1}, batch {i+1}, loss: {dynamic_loss/400}")\n            dynamic_loss = 0.0 \n'

In [11]:
'''
from tqdm import tqdm
pbar = tqdm(trainloader, desc=f"Epoch {epoch+1}")
for batch_idx, (data,target) in enumerate(pbar):
    inputs, labels = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = loss_function(outputs, F.one_hot(labels, num_classes=10).float())
    loss.backward()
    optimizer.step()
    pbar.set_postfix({'loss': loss.item()})
    wandb.log({"train_loss": loss.item()})
    dynamic_loss += loss
    if i%200 ==99:
        print(f"epoch {epoch+1}, batch {i+1}, loss: {dynamic_loss/400}")
        dynamic_loss = 0.0 
'''

'\nfrom tqdm import tqdm\npbar = tqdm(trainloader, desc=f"Epoch {epoch+1}")\nfor batch_idx, (data,target) in enumerate(pbar):\n    inputs, labels = data[0].to(device), data[1].to(device)\n    optimizer.zero_grad()\n    outputs = net(inputs)\n    loss = loss_function(outputs, F.one_hot(labels, num_classes=10).float())\n    loss.backward()\n    optimizer.step()\n    pbar.set_postfix({\'loss\': loss.item()})\n    wandb.log({"train_loss": loss.item()})\n    dynamic_loss += loss\n    if i%200 ==99:\n        print(f"epoch {epoch+1}, batch {i+1}, loss: {dynamic_loss/400}")\n        dynamic_loss = 0.0 \n'

In [12]:
def train(net, device, train_loader, optimizer, epoch):
    net.train()
    # tqdm creates the progress bar
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    train_correct =0
    train_total =0
    for i, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = net(data)
        loss = loss_function(outputs, target)
        loss.backward()
        optimizer.step()
        _, train_predicted = torch.max(outputs, 1)
        train_total += len(target)
        train_correct += (train_predicted == target).sum().item()
        # Update the bar with the latest loss
        pbar.set_description(desc=f'Epoch {epoch} Loss={loss.item():.4f}')
        
        # Log training loss to wandb
        wandb.log({"train_loss": loss.item(), "train_accuracy": 100.0 * train_correct / train_total})
        pbar.set_postfix({'loss': loss.item(), 'train_accuracy': 100.0 * train_correct / train_total})

def test(net, device, testloader):
    net.eval()
    correct_labels = 0
    test_loss = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            test_loss += loss_function(outputs, labels).item()
            #print(f"outputs.data = {outputs.data}")
            _, predicted = torch.max(outputs, 1)
            #if (predicted == labels).sum().item():
            correct_labels += (predicted == labels).sum().item()
    test_loss /= len(testloader.dataset)
    accuracy = 100.0 * correct_labels / len(testloader.dataset)
    wandb.log({"test_accuracy": accuracy, "test_loss": test_loss})
    print(f"\n Test Set: Accuracy: {accuracy:.4f}%, test_loss: {test_loss} \n")
# Log test results to wandb
    return accuracy

In [13]:
'''correct_labels = 0
total_labels = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        probs = F.softmax(outputs, dim =1)
        #print(f"outputs.data = {outputs.data}")
        _, predicted = torch.max(probs, 1)
        total_labels += labels.size(0)
        #if (predicted == labels).sum().item():
        correct_labels += (predicted == labels).sum().item()
        wandb.log({"test_accuracy": 100*correct_labels/total_labels})
print(f"accuracy of the network on the 10000 test images: {100*correct_labels/total_labels} %")
'''

'correct_labels = 0\ntotal_labels = 0\nwith torch.no_grad():\n    for data in testloader:\n        images, labels = data[0].to(device), data[1].to(device)\n        outputs = net(images)\n        probs = F.softmax(outputs, dim =1)\n        #print(f"outputs.data = {outputs.data}")\n        _, predicted = torch.max(probs, 1)\n        total_labels += labels.size(0)\n        #if (predicted == labels).sum().item():\n        correct_labels += (predicted == labels).sum().item()\n        wandb.log({"test_accuracy": 100*correct_labels/total_labels})\nprint(f"accuracy of the network on the 10000 test images: {100*correct_labels/total_labels} %")\n'

In [ ]:
for epoch in range(1, config.epochs + 1):
    train(net, device, trainloader, optimizer, epoch)
    test_accuracy = test(net, device, testloader)
wandb.finish()

Epoch 1 Loss=2.3467:  13%|█▎        | 61/469 [00:09<00:56,  7.22it/s, loss=2.35, train_accuracy=11.4]